<a href="https://colab.research.google.com/github/kaz-kobayashi/kaz-kobayashi/blob/master/190516rinko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

線形最適化問題を解いてみる


最大化 $2x_1 + 3x_2$

条件  $ x_1 + 3x_2 \leq 9$

 $ x_1 + x_2 \leq 4 $

 $ 2x_1 + x_2 \leq 6 $


In [0]:
!pip install pulp

     |████████████████████████████████| 13.6MB 3.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/5e/76/77/e28b22219e46e3b4b033f02e8b36b2770ae545bdcf60c2b224
Successfully built pulp


In [0]:
from pulp import *
prob=LpProblem(name="Lp-Sample",sense=LpMaximize)
x1=LpVariable('x1',lowBound=0.0)
x2=LpVariable('x2',lowBound=0.0)
prob+=2*x1+3*x2 #目的関数の設定
prob+=x1+3*x2<=9,'ineq1'
prob+=x1+x2<=4,'ineq2'
prob+=2*x1+x2<=6,'ineq3'
print(prob)
prob.solve() #求解
print(LpStatus[prob.status])
print("Optimal value=",value(prob.objective))
for v in prob.variables():
  print(v.name,"=",value(v))


Lp-Sample:
MAXIMIZE
2*x1 + 3*x2 + 0
SUBJECT TO
ineq1: x1 + 3 x2 <= 9

ineq2: x1 + x2 <= 4

ineq3: 2 x1 + x2 <= 6

VARIABLES
x1 Continuous
x2 Continuous

Optimal
Optimal value= 10.5
x1 = 1.5
x2 = 2.5


1行目ではpulpパッケージを読み込む．２行目のLpProblemで問題オブジェクトを生成する．name="文字列"で問題に名前をつけることができる．sense=値で，最大化か最小化を選ぶ．最大化ならLpMaximize,最小化ならLpMinimizeを指定する．

3,4行目で，LpVariableを用いて変数x1,x2を定義している．変数の定義の際には，変数の名前，変数の種類（整数かなど）と下限・上限を指定できる．種類はを指定しなければ，連続変数として定義される．

5行目で目的関数を設定している．目的関数は，+=演算子によって線形式を加えることで設定される．複数回実行すると，最後に実行された内容が設定される．例えば，prob+=20*x1+3*x2を実行すると，問題の目的関数は，$ 20x_1+3x_2$ に上書きされる．

In [0]:
prob+=20*x1+3*x2
print(prob)

6, 7, 8行目で制約式を３つ設定している．制約式は，+=演算子によって線形不等式を加えることで設定される（目的関数は線形”式”を追加したこととの違いに注意）．制約式には，名前をつけることができる．

10行目で，この線形最適化問題を解く．解いた結果，最適解が得られたか否かを，LpStatus[prob.status]によって確認することができる．LpStatusは，LPを解いた結果の取りうる状態を格納した辞書であり，prob.status自体はこの辞書のキーとなる値(0,1,-1,-2,-3)を取る．

In [0]:
print(LpStatus)

{0: 'Not Solved', 1: 'Optimal', -1: 'Infeasible', -2: 'Unbounded', -3: 'Undefined'}


12行目で，解いた結果得られた最適値prob.objectiveを表示している．
解いた結果の状態での各変数の値には，13行目，14行目を実行することにより表示される．

##生産計画問題

スーパーSでは，毎日直接農家から３種類の果物，オレンジ，りんご，ぶどうを仕入れて，３種類のミックスジュースA, B, Cを製造・販売している．原料である果物は１日あたりそれぞれオレンジ60kg, りんご36kg, ぶどう48kg仕入れることができる．ミックスジュースAを1リットル作るには，オレンジ，りんごがそれぞれ3kg, 1kg必要で，ミックスジュースBを1リットル作るには，オレンジ，りんご，ぶどうがそれぞれ1kg, 3kg, 2kg必要で，ミックスジュースCを1リットル作るには，オレンジ，ぶどうがそれぞれ2kg, 4kg必要である．製造されたミックスジュースは，1リットルあたりそれぞれ，150円，200円，300円で売れていく．販売利益を最大にするには，３種類の製品を１日あたりどれだけ生産すればよいか ?


|  原料  | 制限  | A |B |C |
| ---- | ---- | ---- |---- |---- |
|  オレンジ  |  60kg  | 3 | 1 | 2 |
|  りんご|  36kg  |1 | 3 | 0 |
|  ぶどう|  48kg  |0 | 2 | 4 |
|  1Lあたりの利益|  ----  |150円| 200円 | 300円 |




最大化 $150x_1 + 200x_2 + 300 x_3$

条件 $ 3x_1+x_2+2x_3 \leq 60 $

$ x_1+3x_2  \leq 36 $

$ 2x_2+4x_3 \leq 48 $


この問題をPuLPで表すために，係数行列，コストベクトル，右側定数ベクトルをあらかじめ定義しておくとよい．そのために，NumPyを使う．


In [0]:
from pulp import *
import numpy as np
A=np.array([[3,1,2],[1,3,0],[0,2,4]])
c=np.array([150,200,300])
b=np.array([60,36,48])
(m,n)=A.shape
print("A=",A)
print("b=",b)
print("c=",c)

A= [[3 1 2]
 [1 3 0]
 [0 2 4]]
b= [60 36 48]
c= [150 200 300]


In [0]:
for i in range(m):
  print(A[i])

[3 1 2]
[1 3 0]
[0 2 4]


In [0]:
prob=LpProblem(name="Production",sense=LpMaximize)
x=[LpVariable('x'+str(i+1),lowBound=0) for i in range(n)]
prob+=lpDot(c,x)
for i in range(m):
  prob += lpDot(A[i],x) <= b[i],'ineq'+str(i)
print(prob)
prob.solve()
print(LpStatus[prob.status])
print('Optimal value =',value(prob.objective))
for v in prob.variables():
  print(v.name,'=',v.varValue)

Production:
MAXIMIZE
150*x1 + 200*x2 + 300*x3 + 0
SUBJECT TO
ineq0: 3 x1 + x2 + 2 x3 <= 60

ineq1: x1 + 3 x2 <= 36

ineq2: 2 x2 + 4 x3 <= 48

VARIABLES
x1 Continuous
x2 Continuous
x3 Continuous

Optimal
Optimal value = 5800.0
x1 = 12.0
x2 = 8.0
x3 = 8.0


全体の流れとしては，前と同じである．まず，LpProblem()によって問題オブジェクトを生成し，変数を生成し，その後目的関数を定めたから制約式を定めている．ここでは，変数はリストxとして生成している．xをリストとして生成し，変数の係数cもベクトルとして定義すると，cとxの内積を生成する関数lpDot(c,x)によって目的関数の線形式を生成することができる．制約式の定義でも同様に内積を生成する関数 LpDot()を用いてる．ここで，Aは3x3の行列を表しており，A[i]は行列Aの第1行目を表す．

こうして定義した線形最適化問題を解くと，最適解(12.0,8.0,8.0)が得られ，最適値は5800である．



### 双対問題の導入

先に計算して得られた最適解$(x_1,x_2,x_3)=(12.0,8.0,8.0)$が実行可能解であることは，次の計算で確認することができる．

In [0]:
X=np.array([v.varValue for v in prob.variables()])
print(X)
np.all(np.abs(b-np.dot(A,X))<=1.0e-5)

[12.  8.  8.]


True

ここで左辺を０ではなく，少し余裕を持たして非常に小さい値 (1e-5)にしているのは，誤差を認めるためである．浮動小数点での計算は丸め誤差が入るため，理論的には（手計算では）０になる値でも，0ちょうどではなくほんの少しズレた値になることが多い．そこで，b-Axが非常に小さな正の値1e-5よりも小さければ，b<=Axと判定することとしているのである．

さて，主問題を

最大化 $ \mathbb{c}^{\top}  \mathbb{x} $ 

条件 $ A\mathbb{x} \leq \mathbb{b}, \mathbb{x} \geq  \mathbb{0} $

と表すと，その双対問題は，

最小化 $ \mathbb{b}^{\top}  \mathbb{y} $ 

条件 $ A^{\top} \mathbb{y} \geq \mathbb{c}, \mathbb{y} \geq  \mathbb{0} $

となる。したがって，先ほど線形最適化問題の定義に用いたベクトルと行列を用いて，双対問題を定義できる。





In [0]:
AT=A.T
dual=LpProblem(name='Dual_Production',sense=LpMinimize)
y=[LpVariable('y'+str(i+1),lowBound=0) for i in range(m)]
dual += lpDot(b,y)
for j in range(n):
  dual += lpDot(AT[j],y)>=c[j], 'ineq'+str(j) 
dual.solve()
print(LpStatus[dual.status])
print('Optimal value of dual problem =',value(dual.objective))
for v in dual.variables():
  print(v.name,"=",v.varValue)

Optimal
Optimal value of dual problem = 5799.999996
y1 = 44.444444
y2 = 16.666667
y3 = 52.777778


こうして，双対問題の最適値は元の線形最適化問題（主問題）の最適値と一致することがわかった．
得られた解yが実行可能解かどうかは，次の計算で確認することができる．



In [0]:
Y=np.array([v.varValue for v in dual.variables()])
np.all(np.abs(np.dot(AT,Y)-c)<=1.0e-5)

True

## A Blending Problem

チキン，ビーフ，マトン，米，小麦，ジェル を用いてキャットフードを生産したい．チキン，ビーフ，マトンの１グラムあたりコストは，それぞれ，0.013ドル，0.008ドル，0.010ドルとする．米，小麦，ジェルの１グラムあたりコストは，それぞれ，0.002ドル，0.005ドル，0.001ドルとする．

各原料は，タンパク質，脂肪，繊維，塩分に寄与する．１グラムあたりに含まれる各栄養素は，次の通りとする．


|  原料  | タンパク質  | 脂肪 |繊維 |塩分 | コスト|
| ---- | ---- | ---- |---- |---- |----|
|  チキン |  0.100  | 0.080 | 0.001 | 0.002 | 0.013 |
|  ビーフ|  0.200  | 0.100 | 0.005 | 0.005 | 0.008 
|  マトン|  0.15 | 0.110 | 0.003 | 0.007 | 0.010 
|  米|  0.000  | 0.010 |  0.100  | 0.002 | 0.002 |
|  小麦|  0.040  | 0.010 |  0.150 | 0.008 | 0.005
|  ジェル|  0.0  | 0.0 |  0.0 | 0.0 | 0.001


それぞれの栄養素の摂取制限は，次のとおりとする．


 | タンパク質（以上）  | 脂肪（以上） |繊維（以下） |塩分（以下） |
| ---- | ---- | ---- |---- |
|  8  |  6  |  2 | 0.4 |


この条件を満たすなかで，かかる費用が最小になるように変数の値を決めたい．

これを線形最適化問題として定式化する．

決定変数は，１缶あたりに入れる各原料の割合とする．こうすると，栄養素に関する制約式は，次のように書かれる:

$ 0.100x_1 + 0.200x_2 + 0.150x_3 + 0.000x_4 + 0.040x_5 + 0.0x_6 \geq 8.0 $

$ 0.080x_1 + 0.100x_2 + 0.110x_3 + 0.010x_4 + 0.010x_5 + 0.0x_6 \geq 6.0 $

$ 0.001x_1 + 0.005x_2 + 0.003x_3 + 0.100x_4 + 0.150x_5 + 0.0x_6 \leq 2.0 $

$ 0.002x_1 + 0.005x_2 + 0.007x_3 + 0.002x_4 + 0.008x_5 + 0.0x_6 \leq 0.4 $

また，目的関数は，次のとおりである．

$ \min  0.013x_1 + 0.008x_2 + 0.010 x_3 + 0.002 x_4 + 0.005 x_5 + 0.001 x_6   $

この問題では，変数の数が６つと，だいぶ多い．６つの原料と４つの栄養素の値を，添字の番号を頼りに誤りなく設定するのが苦しくなってきた．人間がみてもう少しわかりやすいように， Pythonの辞書を用いて値を設定することとする．

まず，栄養素を表すリストを作成する．





In [0]:
Ingredients=["CHICKEN","BEEF","MUTTON","RICE","WHEAT","GEL"]


Ingredientsの各要素（＝原料）をキーとして，コスト，タンパク質，脂肪，繊維，塩分を表す辞書をそれぞれcosts, proteinPercent,fatPercent,fibrePercent,saltPercentとして定義する．



In [0]:
costs = {'CHICKEN': 0.013, 
         'BEEF': 0.008, 
         'MUTTON': 0.010, 
         'RICE': 0.002, 
         'WHEAT': 0.005, 
         'GEL': 0.001}

# A dictionary of the protein percent in each of the Ingredients is created
proteinPercent = {'CHICKEN': 0.100, 
                  'BEEF': 0.200, 
                  'MUTTON': 0.150, 
                  'RICE': 0.000, 
                  'WHEAT': 0.040, 
                  'GEL': 0.000}

# A dictionary of the fat percent in each of the Ingredients is created
fatPercent = {'CHICKEN': 0.080, 
              'BEEF': 0.100, 
              'MUTTON': 0.110, 
              'RICE': 0.010, 
              'WHEAT': 0.010, 
              'GEL': 0.000}

# A dictionary of the fibre percent in each of the Ingredients is created
fibrePercent = {'CHICKEN': 0.001, 
                'BEEF': 0.005, 
                'MUTTON': 0.003, 
                'RICE': 0.100, 
                'WHEAT': 0.150, 
                'GEL': 0.000}

# A dictionary of the salt percent in each of the Ingredients is created
saltPercent = {'CHICKEN': 0.002, 
               'BEEF': 0.005, 
               'MUTTON': 0.007, 
               'RICE': 0.002, 
               'WHEAT': 0.008, 
               'GEL': 0.000}

In [0]:
print(saltPercent)

{'CHICKEN': 0.002, 'BEEF': 0.005, 'MUTTON': 0.007, 'RICE': 0.002, 'WHEAT': 0.008, 'GEL': 0.0}


こうすると，どの数値がどの原料のどの栄養素を表すものかの対応が人間がみてわかりやすくなる．例えば，チキンに含まれる塩分の率は，saltPercent[CHICKEN]などとすればよい．


さて，まず線形最適化問題の問題オブジェクトを生成する．

In [0]:
prob=LpProblem("The Whiskas Problem",LpMinimize)


次に，決定変数オブジェクトを生成するが，ここでは辞書を使った方法を用いる．いま，Ingredientsというリストがあり，このリストの各要素は，原料を表している．そして，決定変数はこの各原料に対して定義したい．そこで，Ingredientsを与えることでその要素それぞれに対して変数を一つずつ定義する命令を用いる．これは，次のようにするとよい:



In [0]:
ingredient_vars=LpVariable.dicts("Ingr",Ingredients,0)
print(type(ingredient_vars))
print(ingredient_vars)
print(type(ingredient_vars['CHICKEN']))

<class 'dict'>
{'CHICKEN': Ingr_CHICKEN, 'BEEF': Ingr_BEEF, 'MUTTON': Ingr_MUTTON, 'RICE': Ingr_RICE, 'WHEAT': Ingr_WHEAT, 'GEL': Ingr_GEL}
<class 'pulp.pulp.LpVariable'>


1行目では，LpVariable.dicts()によって，Ingredientsの各要素をキーとし，それに対応する変数を値とする辞書を生成している．最初の引数は，生成する変数の名前につける接頭辞を指定するものである．２番目の引数は，対応する変数を生成するリストである．こうして生成したingredients_varsの型をtype()で確認すると，確かに辞書 (dict)であることがわかる．また，print(ingredients_vars)によって，キーと値の対応がわかる．例えば，Ingredientsの要素CHIKENに対しては，Ingr_CHICKENという名前の変数が値として生成されている．そして，invredients_varsのキー'CHICKEN'に対する値を ingredients_vars['CHICKEN']で取得し，その型を表示してみると，pulp.pulp.LpVariableとなり，値はLpVariable型であることがわかる． 

こうして定義した辞書ingredient_varsを用いて，目的関数を定義する．目的関数を定義するのに，PuLPで提供されているlpSum()を用いる．これは，引数に与えたリストの和を求めるものである．引数に与えているのは，

[costs[i] * ingredient_vars[i] for i in Ingredients]

というリストである．これは，*内包表記*，と言われる表現である．このリストがどういうものか，中身を print()関数で表示してみると，次のようになる:

In [0]:
print([costs[i] * ingredient_vars[i] for i in Ingredients])


[0.013*Ingr_CHICKEN + 0.0, 0.008*Ingr_BEEF + 0.0, 0.01*Ingr_MUTTON + 0.0, 0.002*Ingr_RICE + 0.0, 0.005*Ingr_WHEAT + 0.0, 0.001*Ingr_GEL + 0.0]


これからわかるように，各要素は，変数に係数がかかったものになっている．例えば，最初の要素は，i=CHICKENに対して，costs['CHICKEN']とingredient_vars['CHICKEN']の積である．辞書costsで，キー'CHICKEN'に対する値costs['CHICKEN']は0.013であるから，0.013*Ingr_CHICKEN + 0.0となっている．このリストの全ての要素の和を求めるのがlpSum()であるから，lpSum([costs[i] * ingredient_vars[i] for i in Ingredients])をprint()で表示すると，次のようになる:

In [0]:
print(lpSum([costs[i] * ingredient_vars[i] for i in Ingredients]))

0.008*Ingr_BEEF + 0.013*Ingr_CHICKEN + 0.001*Ingr_GEL + 0.01*Ingr_MUTTON + 0.002*Ingr_RICE + 0.005*Ingr_WHEAT


こうしてコストを表す線形式が得られるので，それを目的関数として設定する．

In [0]:
prob+=lpSum([costs[i]*ingredient_vars[i] for i in Ingredients]),"Total Cost of Ingredients per can"

同じように，lpSum()を用いて制約式を追加する．Ingredientsの各要素に対して係数をかけて足すことは目的関数の場合と同じであるが，変数にかける係数が栄養素によって異なることに注意する．

In [0]:
prob += lpSum([ingredient_vars[i] for i in Ingredients]) == 100, "PercentagesSum"
prob += lpSum([proteinPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 8.0, "ProteinRequirement"
prob += lpSum([fatPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 6.0, "FatRequirement"
prob += lpSum([fibrePercent[i] * ingredient_vars[i] for i in Ingredients]) <= 2.0, "FibreRequirement"
prob += lpSum([saltPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4, "SaltRequirement"


こうして，キャットフードのコストを最小化する線形最適化問題が定義された． 

In [0]:
print(prob)

The Whiskas Problem:
MINIMIZE
0.008*Ingr_BEEF + 0.013*Ingr_CHICKEN + 0.001*Ingr_GEL + 0.01*Ingr_MUTTON + 0.002*Ingr_RICE + 0.005*Ingr_WHEAT + 0.0
SUBJECT TO
PercentagesSum: Ingr_BEEF + Ingr_CHICKEN + Ingr_GEL + Ingr_MUTTON + Ingr_RICE
 + Ingr_WHEAT = 100

ProteinRequirement: 0.2 Ingr_BEEF + 0.1 Ingr_CHICKEN + 0.15 Ingr_MUTTON
 + 0.04 Ingr_WHEAT >= 8

FatRequirement: 0.1 Ingr_BEEF + 0.08 Ingr_CHICKEN + 0.11 Ingr_MUTTON
 + 0.01 Ingr_RICE + 0.01 Ingr_WHEAT >= 6

FibreRequirement: 0.005 Ingr_BEEF + 0.001 Ingr_CHICKEN + 0.003 Ingr_MUTTON
 + 0.1 Ingr_RICE + 0.15 Ingr_WHEAT <= 2

SaltRequirement: 0.005 Ingr_BEEF + 0.002 Ingr_CHICKEN + 0.007 Ingr_MUTTON
 + 0.002 Ingr_RICE + 0.008 Ingr_WHEAT <= 0.4

VARIABLES
Ingr_BEEF Continuous
Ingr_CHICKEN Continuous
Ingr_GEL Continuous
Ingr_MUTTON Continuous
Ingr_RICE Continuous
Ingr_WHEAT Continuous



これを解くと，最適解が得られる．

In [0]:
prob.solve()
print(LpStatus[prob.status])
print('Optimal value of catfood problem =',value(prob.objective))
for v in prob.variables():
  print(v.name,"=",v.varValue)

Optimal
Optimal value of catfood problem = 0.52
Ingr_BEEF = 60.0
Ingr_CHICKEN = 0.0
Ingr_GEL = 40.0
Ingr_MUTTON = 0.0
Ingr_RICE = 0.0
Ingr_WHEAT = 0.0


##　輸送最適化問題

倉庫群から工場群へ部品を搬送したい．輸送費が最小となる計画を求めたい．

 

*   倉庫群から工場群への輸送量を決めたいので，この量を決定変数とする
*   輸送コストを最小化したいので，これを目的関数とする
*  各倉庫からの搬出は，供給可能量以下とする必要がある．これを最初の制約条件とする．
* 各工場への搬入は，需要量以下としたいので，これを２番目の制約条件とする．







 


|  倉庫   | W1  | 脂肪 |繊維 |塩分 | コスト| コスト|
| ---- | ---- | ---- |---- |---- |----|----|
|  倉庫| W1  | 10 | 10 | 11 | 17  |35|
|  倉庫| W2  | 16 | 19 | 12 | 14 |41|
|  倉庫| W3  | 15 | 12 | 14 | 12 |42 |
|  需要|　需要  | 28 | 29 | 31 | 25 |  |



|  倉庫   | W1  | 脂肪 |繊維 |塩分 | コスト |  コスト|
| ---- | ---- | ---- |---- |---- |----|----|
|  倉庫 | W1  | 10 | 10 | 11 | 17  |35|
